# Model training
The goal of is to train classification model that will predict if the client will subscribe a term deposit. I will train a few models and compare it's results:
1. Models training
    * Linear Regression
    * KNN
    * SVM
    * Decision Tree
    * Random Forest
    * LightGBoost
    * Dense Neutral Network
2. Results comparison
3. Inference


### Model training

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support, classification_report, precision_recall_curve, roc_auc_score, plot_roc_curve
from sklearn.model_selection import validation_curve, GridSearchCV
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Read prepared training and test data
X_train = pd.read_csv("X_train_oversample.csv", index_col=0, header=None).to_numpy()
X_test= pd.read_csv("X_test.csv", index_col=0, header=None).to_numpy()
y_train = pd.read_csv("y_trainoversample.csv", index_col=0, header=None).to_numpy().flatten()
y_test = pd.read_csv("y_test.csv", index_col=0, header=None).to_numpy().flatten()

In [5]:
# Check sizes of read data
print(f"size of X_train: {X_train.shape}")
print(f"size of X_test: {X_test.shape}")
print(f"size of y_train: {y_train.shape}")
print(f"size of y_test: {y_test.shape}")


size of X_train: (65826, 27)
size of X_test: (4119, 27)
size of y_train: (65826,)
size of y_test: (4119,)


It's imporant to properly choose metrics to asses the model. For each model I'll present the following metrics:  accuracy, precision, recall, Fscore, AUC, Confusion Matrix. I do not have knowledge, what are the exact requirements of the Business Case. I will assume that it is important to predict as many customers than will subscribe the product as possible and therefore focuss on choosing model with good recall and overall accuracy.

In [6]:
# Prepare a function presenting results
def results(y_test, y_pred):
  print("Accuracy: {0:0.3f}".format(accuracy_score(y_test, y_pred)))
  print("Precision: {0:0.3f}".format(precision_recall_fscore_support(y_test, y_pred, pos_label=1, average="binary")[0]))
  print("Recall: {0:0.3f}".format(precision_recall_fscore_support(y_test, y_pred, pos_label=1, average="binary")[1]))
  print("Fscore: {0:0.3f}".format(precision_recall_fscore_support(y_test, y_pred, pos_label=1, average="binary")[2]))
  print("AUC: {0:0.3f}".format(roc_auc_score(y_test, y_pred)))
  print("\nConfusion matrix:")
  return pd.crosstab(y_test, y_pred, rownames=["True"], colnames=["Predicted"], margins=True)

### 1.1 Logistic Regression

In [7]:
# Legistic Regression model training
lr = LogisticRegression(random_state=2022).fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
results(y_test, y_pred_lr)

Accuracy: 0.832
Precision: 0.374
Recall: 0.647
Fscore: 0.474
AUC: 0.751

Confusion matrix:


Predicted,0,1,All
True,,,
0,3112,523,3635
1,171,313,484
All,3283,836,4119


### 1.2 K-Nearest Neighbors

In [8]:
# KNN model training
knn = KNeighborsClassifier().fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
results(y_test, y_pred_knn)

Accuracy: 0.787
Precision: 0.293
Recall: 0.572
Fscore: 0.387
AUC: 0.694

Confusion matrix:


Predicted,0,1,All
True,,,
0,2965,670,3635
1,207,277,484
All,3172,947,4119


### 1.3 SVM

In [9]:
sv = svm.SVC(random_state=2022).fit(X_train, y_train)
y_pred_svm = sv.predict(X_test)
results(y_test, y_pred_svm)

Accuracy: 0.843
Precision: 0.390
Recall: 0.595
Fscore: 0.471
AUC: 0.735

Confusion matrix:


Predicted,0,1,All
True,,,
0,3184,451,3635
1,196,288,484
All,3380,739,4119


### 1.4 Decision Tree

In [10]:
# Choose depth of decision tree using greed search
dto = DecisionTreeClassifier(random_state=2022)
param_grid = {'max_depth': [5, 10, 15, 20, 25, 30, 35, 40]}
grid_tree = GridSearchCV(dto, param_grid=param_grid, cv=3, scoring=['accuracy', 'recall'], refit='recall')
grid_tree.fit(X_train, y_train)
for i in np.arange(8):
  print(f"{grid_tree.cv_results_['params'][i]}: mean test accuracy: {grid_tree.cv_results_['mean_test_accuracy'][i]:.3f}, mean test recall: {grid_tree.cv_results_['mean_test_recall'][i]:.3f}")

{'max_depth': 5}: mean test accuracy: 0.759, mean test recall: 0.623
{'max_depth': 10}: mean test accuracy: 0.850, mean test recall: 0.775
{'max_depth': 15}: mean test accuracy: 0.878, mean test recall: 0.824
{'max_depth': 20}: mean test accuracy: 0.889, mean test recall: 0.862
{'max_depth': 25}: mean test accuracy: 0.891, mean test recall: 0.878
{'max_depth': 30}: mean test accuracy: 0.889, mean test recall: 0.880
{'max_depth': 35}: mean test accuracy: 0.890, mean test recall: 0.881
{'max_depth': 40}: mean test accuracy: 0.890, mean test recall: 0.881


##### Comment
Similar results are obtained using depth of tree 30, 35, and 40, therefore I'll choose the simplest model out of those 3- depth 30

In [11]:
# Train model based on chosed max_depth and score it
dt = DecisionTreeClassifier(max_depth=30, random_state=2022)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
results(y_test, y_pred_dt)

Accuracy: 0.845
Precision: 0.335
Recall: 0.326
Fscore: 0.331
AUC: 0.620

Confusion matrix:


Predicted,0,1,All
True,,,
0,3321,314,3635
1,326,158,484
All,3647,472,4119


### 1.5 Random Forest

In [12]:
# Choose parameters for random forest using Grid Search
rf= RandomForestClassifier(random_state=2022)
param_grid = {'max_depth': [6, 8, 10],
              'n_estimators': [20, 50, 100]}
grid_rf = GridSearchCV(rf, param_grid=param_grid, cv=3, scoring=['accuracy', 'recall'], refit='recall')
grid_rf.fit(X_train, y_train)
for i in np.arange(9):
  print(f"{grid_rf.cv_results_['params'][i]}: mean test accuracy: {grid_rf.cv_results_['mean_test_accuracy'][i]:.3f}, mean test recall: {grid_rf.cv_results_['mean_test_recall'][i]:.3f}")

{'max_depth': 6, 'n_estimators': 20}: mean test accuracy: 0.766, mean test recall: 0.667
{'max_depth': 6, 'n_estimators': 50}: mean test accuracy: 0.766, mean test recall: 0.652
{'max_depth': 6, 'n_estimators': 100}: mean test accuracy: 0.765, mean test recall: 0.649
{'max_depth': 8, 'n_estimators': 20}: mean test accuracy: 0.797, mean test recall: 0.699
{'max_depth': 8, 'n_estimators': 50}: mean test accuracy: 0.792, mean test recall: 0.689
{'max_depth': 8, 'n_estimators': 100}: mean test accuracy: 0.787, mean test recall: 0.679
{'max_depth': 10, 'n_estimators': 20}: mean test accuracy: 0.816, mean test recall: 0.726
{'max_depth': 10, 'n_estimators': 50}: mean test accuracy: 0.817, mean test recall: 0.728
{'max_depth': 10, 'n_estimators': 100}: mean test accuracy: 0.815, mean test recall: 0.726


In [13]:
# Train model using chosen hyperparamters
rf = RandomForestClassifier(max_depth=10, n_estimators=50, random_state=2022)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
results(y_test, y_pred_rf)

Accuracy: 0.870
Precision: 0.456
Recall: 0.560
Fscore: 0.503
AUC: 0.736

Confusion matrix:


Predicted,0,1,All
True,,,
0,3312,323,3635
1,213,271,484
All,3525,594,4119


#### 1.6 LightGBoost

In [14]:
# Choose parameters for random forest using Grid Search
lgb = LGBMClassifier(random_state=2022, n_jobs=-1)
param_grid = {'max_depth': [3, 5, 7, 10, 20],
              'n_estimators': [100, 200, 300],
              'learning_rate': [0.1, 0.2, 0.3]}
grid_lgb = GridSearchCV(lgb, param_grid=param_grid, cv=3, scoring=['accuracy', 'recall'], refit='recall')
grid_lgb.fit(X_train, y_train)
grid_lgb.best_params_

{'learning_rate': 0.3, 'max_depth': 20, 'n_estimators': 300}

In [15]:
# Train model using chosen hyperparamter
lgb = LGBMClassifier(max_depth=20, n_estimators=300, learning_rate=0.3, random_state=2022)
lgb.fit(X_train, y_train)
y_pred_lgb = lgb.predict(X_test)
results(y_test, y_pred_lgb)

Accuracy: 0.893
Precision: 0.581
Recall: 0.333
Fscore: 0.423
AUC: 0.650

Confusion matrix:


Predicted,0,1,All
True,,,
0,3519,116,3635
1,323,161,484
All,3842,277,4119


#### 1.7 Dense Neural Network

In [16]:
def sequential_model():
  model = Sequential()
  model.add(Dense(64, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(16, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
 
dn = KerasClassifier(build_fn=sequential_model, epochs=70, batch_size=10, verbose=0)
dn.fit(X_train, y_train)
y_pred_dn = dn.predict(X_test).flatten()
results(y_test, y_pred_dn)

129/129 [==============================] - 1s 3ms/step
Accuracy: 0.776
Precision: 0.274
Recall: 0.550
Fscore: 0.366
AUC: 0.678

Confusion matrix:


Predicted,0,1,All
True,,,
0,2930,705,3635
1,218,266,484
All,3148,971,4119


### Models comparison

In [17]:
# Confusion matrix comparison
dict ={'indicator': ['TN', 'FP', 'FN', 'TP'],
       'lr': confusion_matrix(y_test, y_pred_lr).ravel(),
       'knn': confusion_matrix(y_test, y_pred_knn).ravel(),
       'svm': confusion_matrix(y_test, y_pred_svm).ravel(),
       'dt': confusion_matrix(y_test, y_pred_dt).ravel(),
       'rf': confusion_matrix(y_test, y_pred_rf).ravel(),
       'lgb': confusion_matrix(y_test, y_pred_lgb).ravel(),
       'dn': confusion_matrix(y_test, y_pred_dn).ravel(),}

pd.DataFrame(dict).set_index('indicator')

,lr,knn,svm,dt,rf,lgb,dn
indicator,,,,,,,
TN,3112,2965,3184,3321,3312,3519,2930
FP,523,670,451,314,323,116,705
FN,171,207,196,326,213,323,218
TP,313,277,288,158,271,161,266


In [25]:
# Accuracy comparison
models=[lr, knn, sv, dt, rf, lgb, dn]
pd.Series([model.score(X_test, y_test) for model in models],
    index=['lr', 'svm', 'knn', 'dt', 'rf','lgb', 'dn'], name='Accuracy').sort_values(ascending=False)

lgb    0.893421
rf     0.869871
dt     0.844622
knn    0.842923
lr     0.831513
svm    0.787084
dn     0.775916
Name: Accuracy, dtype: float64

In [26]:
# Precision, recall, Fscore comparison
predictions=[y_pred_lr, y_pred_knn, y_pred_svm, y_pred_dt, y_pred_rf, y_pred_lgb, y_pred_dn]
pd.DataFrame([precision_recall_fscore_support(
    y_test, y_pred, pos_label=1, average="binary")[:3] for y_pred in predictions],
    index=['lr', 'svm', 'knn', 'dt', 'rf', 'lgb', 'dn'], columns = ["precision", "recall", "Fscore"]).sort_values(by='recall', ascending=False)

,precision,recall,Fscore
lr,0.374402,0.646694,0.474242
knn,0.389716,0.595041,0.470973
svm,0.292503,0.572314,0.387142
rf,0.456229,0.559917,0.502783
dn,0.273944,0.549587,0.365636
lgb,0.581227,0.332645,0.423127
dt,0.334746,0.326446,0.330544


#### Summary
The best accuracy was obtained using LightGBoost, but it's recall is only 0.33. As I made assumption that I want to be able to classify properly as many customers that will suscribe the product as possible, I will choose for inference model with both high recall and accuracy- Random Forest. Very simple model Logisic Regression has the best recall but it results in a lot of false positives, it may be also reasonable model to choose as it's very fast, simple and interpretable, but it depends if low precision is acceptable.

### 3. Inference

In [20]:
# Read original data
df = pd.read_csv("bank-additional-full.csv", sep=";")

In [21]:
# Create prediction dataframe
df_test = pd.read_csv("y_test.csv", header=None, index_col=0)
df_test["prediction"] = y_pred_rf
df_test.rename(columns={1: "label"}, inplace=True)

In [22]:
# Join predictions with original data
df_results = df.join(df_test, how="right")

In [23]:
# Show results
df_results

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,label,prediction
0,,,,,,,,,,,,,,,,,,,,,
29971,31,technician,single,university.degree,no,no,yes,cellular,apr,tue,...,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,yes,1,1
34803,30,student,single,unknown,no,no,no,cellular,may,thu,...,0,nonexistent,-1.8,92.893,-46.2,1.266,5099.1,no,0,0
8936,34,services,married,high.school,no,no,no,telephone,jun,thu,...,0,nonexistent,1.4,94.465,-41.8,4.866,5228.1,no,0,0
23565,58,housemaid,married,basic.9y,no,no,no,cellular,aug,thu,...,0,nonexistent,1.4,93.444,-36.1,4.962,5228.1,no,0,0
9902,57,retired,married,basic.4y,unknown,no,no,telephone,jun,wed,...,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,no,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38367,55,entrepreneur,married,professional.course,no,yes,no,cellular,oct,mon,...,0,nonexistent,-3.4,92.431,-26.9,0.739,5017.5,no,0,1
24918,52,technician,married,high.school,no,yes,no,cellular,nov,tue,...,0,nonexistent,-0.1,93.200,-42.0,4.153,5195.8,no,0,0
37634,29,management,single,university.degree,no,no,no,cellular,aug,mon,...,0,nonexistent,-2.9,92.201,-31.4,0.861,5076.2,no,0,1
